In [ ]:
  from keras.datasets import fashion_mnist

from keras.utils import np_utils  
import numpy as np  

import sys
sys.path.append('../')
from util.my_plot import plot_single, plot_batch, show_train_history, plot_images_labels_prediction, plot_image, plot_value_array
  

In [ ]:
# Read MNIST data  
(X_Train, y_Train), (X_Test, y_Test) = fashion_mnist.load_data()  

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
plot_batch(X_Train, y_Train,class_names ,5)
    

In [ ]:
# Translation of data  
X_Train40 = X_Train.reshape(X_Train.shape[0], 28, 28, 1).astype('float32')  
X_Test40 = X_Test.reshape(X_Test.shape[0], 28, 28, 1).astype('float32')  

X_Train40.shape

In [ ]:
y_Train.shape

In [ ]:
plot_single(X_Train40[0].reshape(28,28))

In [ ]:
# Standardize feature data  
X_Train40_norm = X_Train40 / 255  
X_Test40_norm = X_Test40 /255  
plot_single(X_Train40_norm[0].reshape(28,28))

# 建立模型

In [ ]:
from keras.models import Sequential  
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D  
import tensorflow as tf

In [ ]:
# todo:
# 建立 Convolution layer


In [ ]:
# MLP
model.add(Flatten())
model.add(Dense(256, activation=tf.nn.relu, kernel_initializer='normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation=tf.nn.softmax, kernel_initializer='normal'))

In [ ]:
# 定義訓練方式  
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  
model.summary()  

# 訓練模型

In [ ]:
# 開始訓練  
train_history = model.fit(x=X_Train40_norm, y=y_Train, validation_split=0.2,  
                          epochs=10, batch_size=300, verbose=2)

# 以圖形顯示訓練過程

In [ ]:
show_train_history(train_history, 'acc', 'val_acc')  

In [ ]:
show_train_history(train_history, 'loss', 'val_loss')  

# 評估模型準確率

In [ ]:
scores = model.evaluate(X_Test40_norm, y_Test)  
print("\t[Info] Accuracy of testing data = {:2.1f}%".format(scores[1]*100.0))

# 進行預測

In [ ]:
print("\t[Info] Making prediction of X_Test4D_norm")  
prediction = model.predict_classes(X_Test40_norm)  # Making prediction and save result to prediction  
print()  
print("\t[Info] Show 10 prediction result (From 240):")  
print("%s\n" % (prediction[240:250]))

In [ ]:
plot_images_labels_prediction(X_Test, y_Test, prediction, idx=240)  

In [ ]:
import pandas as pd  
print("\t[Info] Display Confusion Matrix:")  
pd.crosstab(y_Test, prediction, rownames=['label'], colnames=['predict'])  

# 針對位移過的圖進行預測

In [ ]:
from util.my_plot import generate_shift_fashion_mnist_data
shift_img, shift_img_label = generate_shift_fashion_mnist_data(10000)
plot_batch(shift_img, shift_img_label,class_names ,5)


In [ ]:
shift_img = shift_img.reshape(shift_img.shape[0], 28, 28, 1).astype('float32')  
aa_norm = shift_img /255  

scores = model.evaluate(aa_norm, shift_img_label)
print("Accuracy of generated shift data = {:2.1f}%".format(scores[1]*100.0))